[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nakamura196/ddc-tutorial2025/blob/main/notebooks/tutorial_01_rawgraphs/fetch_ndc_labels.ipynb)

# NDC9 ラベル取得スクリプト

ジャパンサーチ SPARQL エンドポイントから NDC9（日本十進分類法 第9版）のラベルを取得します。

- データソース: https://jpsearch.go.jp/rdf/sparql/
- NDC URI形式: `http://jla.or.jp/data/ndc9#{code}`

## 設定

取得するNDCクラスの接頭辞を指定します（例: "91" で日本文学）

In [ ]:
# 取得するNDCクラスの接頭辞（変更可能）
CLASS_PREFIX = "91"  # 91: 日本文学、8: 言語、など

## ライブラリのインポート

In [ ]:
import csv
import json
import urllib.parse
import urllib.request
from io import StringIO

import pandas as pd  # Google Colabにはプリインストール済み

## SPARQL クエリの実行

In [ ]:
SPARQL_ENDPOINT = "https://jpsearch.go.jp/rdf/sparql/"

def fetch_ndc_hierarchical(class_prefix: str) -> list[dict]:
    """
    SPARQLで階層情報付きのNDCラベルを一括取得
    
    Args:
        class_prefix: NDCクラスの接頭辞（例: "91", "8"）
    
    Returns:
        [{code, label_ja, label_hierarchical}, ...] のリスト
    """
    query = f"""PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
SELECT ?code ?labelJa ?parentCode ?parentLabel ?grandCode ?grandLabel
WHERE {{
  ?uri skos:notation ?code ;
       skos:prefLabel ?labelJa .
  FILTER(STRSTARTS(?code, '{class_prefix}'))
  FILTER(LANG(?labelJa) = 'ja')
  OPTIONAL {{
    ?uri skos:broader ?parent .
    ?parent skos:notation ?parentCode ;
            skos:prefLabel ?parentLabel .
    FILTER(LANG(?parentLabel) = 'ja')
    OPTIONAL {{
      ?parent skos:broader ?grand .
      ?grand skos:notation ?grandCode ;
             skos:prefLabel ?grandLabel .
      FILTER(LANG(?grandLabel) = 'ja')
    }}
  }}
}}
ORDER BY ?code"""

    url = SPARQL_ENDPOINT + "?" + urllib.parse.urlencode({
        'query': query,
        'output': 'json'
    })

    req = urllib.request.Request(url, headers={'User-Agent': 'Python/3'})
    with urllib.request.urlopen(req, timeout=30) as res:
        data = json.loads(res.read().decode('utf-8'))

    results = []
    for r in data['results']['bindings']:
        code = r['code']['value']
        label = r['labelJa']['value']
        parent_label = r.get('parentLabel', {}).get('value', '')
        grand_label = r.get('grandLabel', {}).get('value', '')

        # 階層ラベルを構築
        parts = []
        if grand_label:
            parts.append(grand_label)
        if parent_label:
            parts.append(parent_label)
        parts.append(label)

        # 重複を除去（親と同じラベルの場合）
        unique_parts = []
        for p in parts:
            if not unique_parts or p != unique_parts[-1]:
                unique_parts.append(p)

        hierarchical = '/'.join(unique_parts)

        results.append({
            'code': code,
            'label_ja': label,
            'label_hierarchical': hierarchical
        })

    return results

## NDCラベルの取得

In [ ]:
print(f"NDC9 {CLASS_PREFIX}類のラベルを取得中...")

results = fetch_ndc_hierarchical(CLASS_PREFIX)

print(f"取得件数: {len(results)}件")

## DataFrameに変換して表示

In [ ]:
df = pd.DataFrame(results)
df

## CSVファイルとしてダウンロード（Google Colab用）

In [ ]:
# Google Colabでファイルをダウンロード
try:
    from google.colab import files
    
    filename = f"ndc9_class{CLASS_PREFIX}_hierarchical.csv"
    df.to_csv(filename, index=False, encoding='utf-8')
    files.download(filename)
    print(f"{filename} をダウンロードしました")
except ImportError:
    # ローカル環境の場合
    filename = f"ndc9_class{CLASS_PREFIX}_hierarchical.csv"
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f"{filename} を保存しました")

## サンプル表示

In [ ]:
print("\nサンプル（最初の10件）:")
for _, row in df.head(10).iterrows():
    print(f"  {row['code']}: {row['label_hierarchical']}")